In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

from pandas import set_option
set_option("display.max_rows", 20)
pd.options.mode.chained_assignment = None

filename = 'data/facies_vectors.csv'
training_data = pd.read_csv(filename)
training_data.describe()

,Facies,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
count,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,3232.000000,4149.000000,4149.000000
mean,4.503254,2906.867438,64.933985,0.659566,4.402484,13.201066,3.725014,1.518438,0.521852
std,2.474324,133.300164,30.302530,0.252703,5.274947,7.132846,0.896152,0.499720,0.286644
min,1.000000,2573.500000,10.149000,-0.025949,-21.832000,0.550000,0.200000,1.000000,0.000000
25%,2.000000,2821.500000,44.730000,0.498000,1.600000,8.500000,NaN,1.000000,0.277000
50%,4.000000,2932.500000,64.990000,0.639000,4.300000,12.020000,NaN,2.000000,0.528000
75%,6.000000,3007.000000,79.438000,0.822000,7.500000,16.050000,NaN,2.000000,0.769000
max,9.000000,3138.000000,361.150000,1.800000,19.312000,84.400000,8.094000,2.000000,1.000000


In [2]:
grouped = training_data.groupby(['Well Name'])

### At this point we create new variable representing the absolute distance of a deposit to an upper marine context (younger)

In [3]:
new_vars = pd.DataFrame()

for key in grouped.groups.keys():
    
    NM_M = grouped.get_group(key)['NM_M'].values
    
    temp_df = pd.DataFrame()
    temp_df['Depth'] = grouped.get_group(key)['Depth']
    temp_df['Well Name'] = [key for _ in range(len(NM_M))]
        
    dh_2 = np.zeros(len(NM_M))
    count2 = -99999
#     count2=0
    
    for i in range(len(NM_M)):
               
        if ((NM_M[i] == 1) & (count2>-99999)):
            
            count2+=0.5
            dh_2[i] += count2
            
        elif NM_M[i] == 2:
            
            count2=0
        
        else:
            dh_2[i] = count2

    temp_df['dh_2'] = dh_2
        
    new_vars = new_vars.append(temp_df)

### We merge our new variable with the training_data Dataframe

In [4]:
new_vars = new_vars.sort_index()
new_vars =new_vars.drop(['Well Name','Depth'],axis=1)
training_data =  pd.concat([training_data,new_vars],axis=1)
training_data

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,dh_2
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000,-99999.0
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979,-99999.0
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957,-99999.0
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936,-99999.0
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915,-99999.0
5,3,A1 SH,SHRIMPLIN,2795.5,73.970,0.636,14.000,13.385,3.600,1,0.894,-99999.0
6,3,A1 SH,SHRIMPLIN,2796.0,73.720,0.630,15.600,13.930,3.700,1,0.872,-99999.0
7,3,A1 SH,SHRIMPLIN,2796.5,75.650,0.625,16.500,13.920,3.500,1,0.830,-99999.0
8,3,A1 SH,SHRIMPLIN,2797.0,73.790,0.624,16.200,13.980,3.400,1,0.809,-99999.0
9,3,A1 SH,SHRIMPLIN,2797.5,76.890,0.615,16.900,14.220,3.500,1,0.787,-99999.0


### We create a second variable that gives the absolute distance of non-marine sediments to an older (deeper) marine episode

In [5]:
new_vars = pd.DataFrame()

for key in grouped.groups.keys():
    
    NM_M = grouped.get_group(key)['NM_M'].values
    
    temp_df = pd.DataFrame()
    temp_df['Depth'] = grouped.get_group(key)['Depth']
    temp_df['Well Name'] = [key for _ in range(len(NM_M))]
        
    db_2 = np.zeros(len(NM_M))
    count2 = -99999
#     count2 = 0
    
    for i in range(len(NM_M)-1,-1,-1):
               
        if ((NM_M[i] == 1) & (count2>-99999)):
            
            count2+=0.5
            db_2[i] += count2
            
        elif NM_M[i] == 2:            
            count2=0

        else:
            db_2[i] = count2
            
    temp_df['db_2'] = db_2
        
    new_vars = new_vars.append(temp_df)

### We again merge this new variable with our Dataframe

In [6]:
new_vars = new_vars.sort_index()
new_vars = new_vars.drop(['Well Name','Depth'],axis=1)
training_data =  pd.concat([training_data,new_vars],axis=1)
training_data

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,dh_2,db_2
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000,-99999.0,21.5
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979,-99999.0,21.0
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957,-99999.0,20.5
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936,-99999.0,20.0
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915,-99999.0,19.5
5,3,A1 SH,SHRIMPLIN,2795.5,73.970,0.636,14.000,13.385,3.600,1,0.894,-99999.0,19.0
6,3,A1 SH,SHRIMPLIN,2796.0,73.720,0.630,15.600,13.930,3.700,1,0.872,-99999.0,18.5
7,3,A1 SH,SHRIMPLIN,2796.5,75.650,0.625,16.500,13.920,3.500,1,0.830,-99999.0,18.0
8,3,A1 SH,SHRIMPLIN,2797.0,73.790,0.624,16.200,13.980,3.400,1,0.809,-99999.0,17.5
9,3,A1 SH,SHRIMPLIN,2797.5,76.890,0.615,16.900,14.220,3.500,1,0.787,-99999.0,17.0


### Since we have an unbalanced dataset (facies proportions) we call the imblearn library to do some oversampling of the minority classes

In [7]:
from imblearn.over_sampling import SMOTE

### We also import the ensemble module from sklearn and the metrics module to calculate

In [8]:
from sklearn import metrics
from sklearn import ensemble

In [9]:
#Assign category integer to Formation
Formation_list=np.unique(training_data.Formation)
training_data['Formation_category']=0
print (Formation_list)
for f,i in zip(Formation_list,range(len(Formation_list))):
    print(f)
    idx = (training_data.Formation == f)
    training_data.Formation_category[idx]=i
training_data.describe()
    

['A1 LM' 'A1 SH' 'B1 LM' 'B1 SH' 'B2 LM' 'B2 SH' 'B3 LM' 'B3 SH' 'B4 LM'
 'B4 SH' 'B5 LM' 'B5 SH' 'C LM' 'C SH']
A1 LM
A1 SH
B1 LM
B1 SH
B2 LM
B2 SH
B3 LM
B3 SH
B4 LM
B4 SH
B5 LM
B5 SH
C LM
C SH


,Facies,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,dh_2,db_2,Formation_category
count,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,3232.000000,4149.000000,4149.000000,4149.000000,4149.00000,4149.000000
mean,4.503254,2906.867438,64.933985,0.659566,4.402484,13.201066,3.725014,1.518438,0.521852,-9781.547481,5.09677,6.434322
std,2.474324,133.300164,30.302530,0.252703,5.274947,7.132846,0.896152,0.499720,0.286644,29716.387851,7.75078,4.722771
min,1.000000,2573.500000,10.149000,-0.025949,-21.832000,0.550000,0.200000,1.000000,0.000000,-99999.000000,0.00000,0.000000
25%,2.000000,2821.500000,44.730000,0.498000,1.600000,8.500000,NaN,1.000000,0.277000,0.000000,0.00000,2.000000
50%,4.000000,2932.500000,64.990000,0.639000,4.300000,12.020000,NaN,2.000000,0.528000,0.000000,0.00000,7.000000
75%,6.000000,3007.000000,79.438000,0.822000,7.500000,16.050000,NaN,2.000000,0.769000,5.500000,8.50000,12.000000
max,9.000000,3138.000000,361.150000,1.800000,19.312000,84.400000,8.094000,2.000000,1.000000,46.000000,46.00000,13.000000


In [10]:
id_NM = training_data.NM_M == 1
id_M = training_data.NM_M == 2

NM_training_data = training_data[id_NM]
M_training_data = training_data[id_M]


NM_training_data.replace(to_replace=np.nan,value=-99999,inplace=True)
M_training_data.replace(to_replace=np.nan,value=-99999,inplace=True)

NM_training_data.describe()

,Facies,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,dh_2,db_2,Formation_category
count,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.0,1998.000000,1998.000000,1998.000000,1998.000000
mean,2.314314,2867.824825,74.877176,0.530167,5.373860,16.760651,-18866.134227,1.0,0.511736,-20312.132382,10.583834,6.546046
std,0.854635,133.370818,15.233573,0.160396,6.147445,7.699085,39136.744191,0.0,0.288464,40251.953018,8.165698,4.357650
min,1.000000,2573.500000,26.060000,-0.025949,-19.900000,3.950000,-99999.000000,1.0,0.000000,-99999.000000,0.500000,0.000000
25%,2.000000,2776.625000,64.378000,0.438000,2.700000,11.950000,2.486000,1.0,0.259250,1.000000,4.000000,3.000000
50%,2.000000,2893.750000,73.704000,0.542000,5.800000,14.908750,3.057000,1.0,0.500000,6.000000,9.000000,7.000000
75%,3.000000,2969.000000,83.936000,0.633000,9.300000,19.212750,3.364750,1.0,0.763000,12.000000,15.000000,11.000000
max,8.000000,3111.500000,183.375000,0.966000,19.257000,84.400000,5.100000,1.0,1.000000,46.000000,46.000000,13.000000


In [11]:
nm_drop_list = ['Formation', 'Well Name', 'Depth',
                                     'Facies','DeltaPHI','PE','NM_M','RELPOS']
m_drop_list = ['Formation', 'Well Name', 'Depth',
                                      'Facies','db_2','dh_2','PE','NM_M','PHIND']

NM_training_data.drop(nm_drop_list,axis=1)

,GR,ILD_log10,PHIND,dh_2,db_2,Formation_category
0,77.450,0.664,11.915,-99999.0,21.5,1
1,78.260,0.661,12.565,-99999.0,21.0,1
2,79.050,0.658,13.050,-99999.0,20.5,1
3,86.100,0.655,13.115,-99999.0,20.0,1
4,74.580,0.647,13.300,-99999.0,19.5,1
5,73.970,0.636,13.385,-99999.0,19.0,1
6,73.720,0.630,13.930,-99999.0,18.5,1
7,75.650,0.625,13.920,-99999.0,18.0,1
8,73.790,0.624,13.980,-99999.0,17.5,1
9,76.890,0.615,14.220,-99999.0,17.0,1


In [12]:
clf = ensemble.RandomForestClassifier(n_estimators=500,n_jobs=-1)

for _ in range(1):
    names = list(np.unique(training_data['Well Name']))

    nm_grouped = NM_training_data.groupby(['Well Name'])
    m_grouped = M_training_data.groupby(['Well Name'])
    
    new_df = pd.DataFrame()

    scores = []

    for name in names:
        
        temp_df = pd.DataFrame()

        if name == 'Recruit F9':

            #Build list of well names and remove blind well (and remove recruit F9 for NM)
            m_train_names = names.copy()
            m_train_names.remove(name)

            # Do it for marine sediments
            m_test = m_grouped.get_group(name)
            m_test_depth = m_test.Depth
            m_X_test = m_test.drop(m_drop_list, axis=1).values
            y_test = m_test['Facies'].values

            m_train = pd.DataFrame()

            for train_name in m_train_names:
                m_train = m_train.append(m_grouped.get_group(train_name))

            id_train_M = m_train.Facies >= 4
            m_train = m_train [id_train_M]
            m_X_train = m_train.drop(m_drop_list, axis=1).values
            m_y_train = m_train['Facies'].values

            #The prediction begins

            #We start with oversampling minority classes
            smt = SMOTE()
            m_X_train, m_y_train = smt.fit_sample(m_X_train,m_y_train)

            #Then we do random forest classification
            clf.fit(m_X_train, m_y_train)
            y_pred = clf.predict(m_X_test)

        else:

            #Build list of well names and remove blind well (and remove recruit F9 for NM)
            m_train_names = names.copy()
            m_train_names.remove(name)   

            nm_train_names = m_train_names.copy()
            nm_train_names.remove('Recruit F9')


            # Do it for non-marine sediments
            nm_test = nm_grouped.get_group(name)
            nm_test_depth = nm_test.Depth
            nm_X_test = nm_test.drop(nm_drop_list, axis=1).values
            nm_y_test = nm_test['Facies'].values

            nm_train = pd.DataFrame()

            for train_name in nm_train_names:
                nm_train = nm_train.append(nm_grouped.get_group(train_name))

            id_train_NM = nm_train.Facies <= 3
            nm_train = nm_train [id_train_NM]
            nm_X_train = nm_train.drop(nm_drop_list, axis=1).values
            nm_y_train = nm_train['Facies'].values

            #The prediction begins

            #We start with oversampling minority classes
            smt = SMOTE()
            nm_X_train, nm_y_train = smt.fit_sample(nm_X_train,nm_y_train)

            #Then we do random forest classification
            clf.fit(nm_X_train, nm_y_train)
            nm_y_pred = clf.predict(nm_X_test)
            
            print(clf.feature_importances_)

            #*********************************************************************#

            # Do it for marine sediments
            m_test = m_grouped.get_group(name)
            m_test_depth = m_test.Depth
            m_X_test = m_test.drop(m_drop_list, axis=1).values
            m_y_test = m_test['Facies'].values

            m_train = pd.DataFrame()

            for train_name in m_train_names:
                m_train = m_train.append(m_grouped.get_group(train_name))

            id_train_M = m_train.Facies >= 4
            m_train = m_train [id_train_M]
            m_X_train = m_train.drop(m_drop_list, axis=1).values
            m_y_train = m_train['Facies'].values

            #The prediction begins

            #We start with oversampling minority classes
            smt = SMOTE()
            m_X_train, m_y_train = smt.fit_sample(m_X_train,m_y_train)

            #Then we do random forest classification
            clf.fit(m_X_train, m_y_train)
            m_y_pred = clf.predict(m_X_test)
            
            print(clf.feature_importances_)

            #================================================================#
            #                         combine results
            #================================================================#

            y_test = np.hstack((nm_y_test,m_y_test))
            y_pred = np.hstack((nm_y_pred,m_y_pred))

        #Scoring
        conf_mat = metrics.confusion_matrix(y_test,y_pred)
        print(conf_mat)
        try:
            score = metrics.f1_score(y_test, y_pred,average='weighted')
        except:
            score = conf_mat[1,1]/np.sum(conf_mat)
        scores.append(score)
        print('********')
        print('Blind well is {0}, F1 score : {1:.4%}\n'.format(name,score))
        
        if name == 'Recruit F9':
            depth = m_test_depth
        else:
            depth = np.hstack((nm_test_depth,m_test_depth))
            
        idx = np.argsort(depth)
        temp_df['Depth'] = depth[idx]
        temp_df['True Facies'] = y_test[idx]
        temp_df['Predicted Facies'] = y_pred[idx]
        
        temp_df['Well Name'] = [name for _ in range(len(depth))]
        new_df = new_df.append(temp_df)

    print("="*30)
    print('*********** RESULT ***********')
    print("="*30)
    print('\nAverage  F1-score is {:.4%}'.format(np.mean(scores)))

[ 0.14816286  0.20281003  0.15978199  0.1727083   0.19246085  0.12407597]
[ 0.21286938  0.2701559   0.19590708  0.20613155  0.1149361 ]
[[96 21  0  0  0  0  0  0]
 [41 50  0  0  0  0  0  0]
 [ 0  0 33  0  2  3  6  0]
 [ 0  0  4 11  2  1  8  0]
 [ 0  0 14 11 14 10 20  0]
 [ 0  0  0  4  0 12  0  0]
 [ 0  0  6  5 10 22 50  5]
 [ 0  0  0  0  0  0  0  5]]
********
Blind well is ALEXANDER D, F1 score : 57.1885%

[ 0.1593502   0.19630373  0.15981984  0.17648327  0.18848361  0.11955935]
[ 0.21959168  0.28357631  0.20081484  0.20484485  0.09117232]
[[ 0  4  4  0  0  0  0  0  0]
 [ 0 33 22  0  0  1  0  0  0]
 [ 0  5 39  0  0  4  0  3  0]
 [ 0  0  0  4  0  8  0  1  0]
 [ 0  0  1  1  7 17  0  4  0]
 [ 0  0  0  6  0 53  4 24  0]
 [ 0  0  1  0  3  2  3 20  5]
 [ 0  1  2  0  1 19  1 50  1]
 [ 0  0  0  0  0  0  0  0 50]]
********
Blind well is CHURCHMAN BIBLE, F1 score : 56.3721%

[ 0.18720172  0.15684316  0.16944057  0.18597613  0.1755302   0.12500822]
[ 0.2172562   0.26610912  0.19657447  0.21076049

In [13]:
new_df.to_pickle('Facies_prediction_ml_contest.pkl')